In [ ]:

from bs4 import BeautifulSoup as bs
from bs4 import Comment
import pandas as pd
import csv
import os
import shutil
import glob
from pathlib import Path


In [ ]:
directory = '/home/inga/Documents/mdrs_data_raw/s1-9/MDRS_CrewReports_FS01-FS09'

In [ ]:
def get_text(doc_path):
    if '.txt' in str(doc_path):
        with open (doc_path, 'rb') as file:
            text = file.read()
        return text
            
    elif '.asp' in str(doc_path) or '.html' in str(doc_path):
        with open (doc_path, 'rb') as file:
            soup = bs(file, 'lxml')
            
        if soup.find("div", {"id": "content"}):
            content = soup.find("div", {"id" : "content"})
            text = content.get_text()
            return text   
        else:
            html = []
            for comment in soup.find_all(string=lambda text:isinstance(text, Comment)):
                if comment.strip() == 'Begin Content':
                    next_node = comment.next_sibling
                    while next_node and next_node.next_sibling:
                        html.append(next_node)
                        next_node = next_node.next_sibling
                        if not next_node.name and next_node.strip() == 'End Content': break;
        

            soup1 = bs(str(html), 'lxml')
            text = soup1.get_text()
            return text

In [ ]:
results = []    
season = []
date = []
name = []
body = []
for path in Path(directory).rglob('*.*'):
    if '.txt' in str(path) or '.asp' in str(path) or '.html' in str(path):
        season.append(str(path).split('/')[-3])
        date.append(str(path).split('/')[-2])
        name.append(str(path).split('/')[-1].split('.')[0])
        body.append(get_text(path))

In [ ]:
reports = pd.DataFrame({'season': season, 'date': date, 'header': name, 'body': body })
reports.to_csv(r's1-9.csv')